In [1]:
# basic packages
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn

#Shared/Utility scripts
import sys
sys.path.insert(0, '../..') #sys allows for the .ipynb file to connect to the shared folder files
from shared_scripts import Simple_Eval, dataloader, mlp_dataprocessing, mlp_model


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HOME = os.path.expanduser('~')


modelname = 'MLP'
model_path = f"{HOME}/NWM_ML/Model/{modelname}"
print(f"{modelname} development script")

Device: cuda
Device: cuda
MLP development script


In [2]:
modelname = 'MLP'
model_path = f"{HOME}/NWM_ML/Model/{modelname}"

cfsday_AFday = 1.983

#input columns
input_columns =[
                'Lat', 
                'Long', 
                'Drainage_area_mi2', 
                'Mean_Basin_Elev_ft',       
                'Perc_Forest', 
                'Perc_Develop', 
                'Perc_Imperv', 
                'Perc_Herbace',       
                'Perc_Slop_30', 
                'Mean_Ann_Precip_in', 
                's1',       
                's2', 
                'storage', 
                'swe', 
                'NWM_flow', 
                'DOY', 
                'tempe(F)', 
                'precip(mm)'
                ]

target = 'flow_cfs'

test_years = [2019, 2020]                 

#load data
datapath = f"{HOME}/NWM_ML/Data/input"
trainingfile = "final_input.parquet"

df, StreamStats = dataloader.get_ML_Data(datapath, trainingfile)
df.head()

df needs no processing


,station_id,Lat,Long,Drainage_area_mi2,Mean_Basin_Elev_ft,Perc_Forest,Perc_Develop,Perc_Imperv,Perc_Herbace,Perc_Slop_30,...,datetime,flow_cfs,s1,s2,storage,swe,NWM_flow,DOY,tempe(F),precip(mm)
0,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-28,78.55521,-0.891007,-0.453991,0.0,1.2,55.0,301,39.239582,0.0
1,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-29,98.61146,-0.891007,-0.453991,0.0,1.2,55.0,302,45.068712,0.0
2,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-30,97.60208,-0.891007,-0.453991,0.0,1.1,54.0,303,50.945891,0.0
3,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-10-31,99.33125,-0.891007,-0.453991,0.0,1.2,54.0,304,45.480097,0.0
4,10011500,40.965225,-110.853508,174.0,9720.0,67.7,1.2,0.12,2.94,27.2,...,2010-11-01,95.76354,-0.998630,0.052336,0.0,1.2,54.0,305,46.656777,0.0


### Dataprocessing
* Editing the features based on the feature importance
* Remove headwater stations from dataset
* make sure dates are in datetime format

In [3]:
#get non headwater stations
headwater_stations = ['10011500', # Bear River headwaters before WY state line
                      '10109000', # Logan River above dams
                      '10113500', # HW Blacksmith fork
                      '10128500', # Upper Weber above Oakley
                      '10131000', #Chalk creek before Weber - lots of upstream irrigation, potentially include
                        '10146400', #Currant Creek above Mona Reservoir - lots of upstream irrigation, potentially include
                        '10150500', #Spanish fork after diamond fork - potentially include because of 6th water diversion CUP
                        '10154200', #Upper Provo river after confluence of N/S forks - potentially include because of duchense tunnel water diversion CUP
                        '10172700', #Vernon creek 2 ranges west of Utah Lake, shouldnt be included because not in GSL basin 
                        '10172800', #Willow creek west of Gransville,  shouldnt be included because does not make it to GSL
                          '10172952'
                          ] #Dunn creek in Raft River Range, shouldnt be included because drains to bonnevile salt flats 

#remove headwater stations
df = df[~df['station_id'].isin(headwater_stations)]

#get stations with correct swe and storage features
#The following sites have swe 

'''
['10011500', '10105900', '10109000', '10126000', '10131000',
       '10133650', '10133800', '10133980', '10134500', '10136500',
       '10140700', '10141000', '10150500', '10154200', '10155000',
       '10155200']
'''

#the following sites have swe and storage
'''
['10126000', '10134500', '10136500', '10140700', '10141000',
       '10155200']
'''

stations = df['station_id'][(df['swe']>0) & (df['storage']>0)].unique()

#Train model with these stations
df = df[df['station_id'].isin(stations)]

#convert dates to datetime format
df.datetime = pd.to_datetime(df.datetime)

# #reset index to clean up df
df.reset_index( inplace =  True, drop = True)

#fit a scaler,save, and scale the training data
x_train_scaled, y_scaled_train = mlp_dataprocessing.mlp_scaler(
                                                      df,
                                                      test_years, 
                                                      target, 
                                                      input_columns, 
                                                      model_path, 
                                                      scalertype = 'MinMax'
                                                      )

x_test_scaled, y_test_temp, x_test_temp, station_index_list = mlp_dataprocessing.mlp_testscaler(
                                                      df,
                                                      test_years, 
                                                      target, 
                                                      input_columns, 
                                                      model_path, 
                                                      )

# Convert to tensor for PyTorch
x_train_scaled_t = torch.Tensor(x_train_scaled)
y_train_scaled_t = torch.Tensor(y_scaled_train)
#Make sure the tensors on are the respective device (cpu/gpu)
x_train_scaled_t = x_train_scaled_t.to(device)
y_train_scaled_t = y_train_scaled_t.to(device)

(26089, 1)
(26089, 18)
(3834, 18)


### Set up Testing year
* Select year(s) not used in training
* Convert to numpy array
* Load scaler and scale data

## Train the model

* randomize training data..
* add training loss https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/

## Loss functions

### Mean Absolute Error (MAE)
Regression problems, especially when the distribution of the target variable has outliers, such as small or big values that are a great distance from the mean value.
It is considered to be more robust to outliers.
PyTorch implementation as **nn.L1Loss()**

### Mean Squared Error (MSE)
The MSE, also called L2 Loss, computes the average of the squared differences between actual values and predicted values.
Pytorch MSE Loss always outputs a positive result, regardless of the sign of actual and predicted values. 
To enhance the accuracy of the model, you should try to reduce the L2 Loss—a perfect value is 0.0. 

The squaring implies that larger mistakes produce even larger errors than smaller ones. 
If the classifier is off by 100, the error is 10,000. If it’s off by 0.1, the error is 0.01. This punishes the model for making big mistakes and encourages small mistakes. 

MSE is the default loss function for most Pytorch regression problems.

### Make your own loss function
https://neptune.ai/blog/pytorch-loss-functions

In [4]:
#Train the model
# Hyperparameters
epochs = 75 # - seems to converge around 80 epochs with 100 batrch size
batch_size = 100
learning_rate = 0.0001  #look up learning rate scheduler https://www.geeksforgeeks.org/understanding-pytorch-learning-rate-scheduling/ -  smaller learning rates doing better!
decay = 0.0005
L1 = 128
L2 = 128
L3 = 64
L4 = 64
L5 = 32
L6 = 16
layers = x_train_scaled_t.shape[1], L1, L2, L3, L4, L5, L6
params =  learning_rate, decay, epochs, batch_size
loss_func = nn.MSELoss()

#Train the model
mlp_model.mlp_train(x_train_scaled_t,
                    y_train_scaled_t, 
                    layers, params, 
                    loss_func, 
                    model_path, 
                    modelname, 
                    shuffle = True)


#Make a prediction for each location, save as compressed pkl file, and send predictions to AWS for use in CSES
Preds_Dict = mlp_model.mlp_predict(test_years, 
                      layers, 
                      model_path, 
                      modelname, 
                      stations, 
                      x_test_temp,
                      x_test_scaled, 
                      y_test_temp,
                      StreamStats,
                      station_index_list)

#Evaluate model performance of the different models, 'flow_cfs_pred', 
prediction_columns = ['NWM_flow', f"{modelname}_flow"]
Eval_DF = Simple_Eval.Simple_Eval(Preds_Dict, 
                                  prediction_columns, 
                                  modelname, 
                                  supply = False,
                                  plots = False, 
                                  keystats = False        
                                  )
Eval_DF



Epoch 1/75, Loss: 0.02311498073524692
Epoch 2/75, Loss: 0.0030765285716798886
Epoch 3/75, Loss: 0.002456707395729163
Epoch 4/75, Loss: 0.00231380260440653
Epoch 5/75, Loss: 0.0022269855182032706
Epoch 6/75, Loss: 0.0021210888850500885
Epoch 7/75, Loss: 0.0020384368241605816
Epoch 8/75, Loss: 0.0019672278304124014
Epoch 9/75, Loss: 0.001870048827834256
Epoch 10/75, Loss: 0.0017845449605430352
Epoch 11/75, Loss: 0.001673668489717411
Epoch 12/75, Loss: 0.0015684880752926443
Epoch 13/75, Loss: 0.0015116746595260505
Epoch 14/75, Loss: 0.0014267464388488397
Epoch 15/75, Loss: 0.00134658229721744
Epoch 16/75, Loss: 0.0012770425221638334
Epoch 17/75, Loss: 0.0012359107334712714
Epoch 18/75, Loss: 0.0011636838022533846
Epoch 19/75, Loss: 0.0011225935934562864
Epoch 20/75, Loss: 0.0010824542032876784
Epoch 21/75, Loss: 0.0010506976692459165
Epoch 22/75, Loss: 0.0010234199008138107
Epoch 23/75, Loss: 0.000985193576731084
Epoch 24/75, Loss: 0.000997334886803934
Epoch 25/75, Loss: 0.000973272468135

,NWM_flow_kge,MLP_flow_kge,NWM_flow_rmse,MLP_flow_rmse,NWM_flow_mape,MLP_flow_mape,NWM_flow_pbias,MLP_flow_pbias
station_id,,,,,,,,
10134500,-0.97,0.58,131.865461,38.674050,545.15,160.31,-173.33,-31.62
10140700,0.29,0.48,236.421011,251.799331,221.61,162.13,-30.30,-6.89
10136500,-0.42,0.44,644.133791,370.697261,330.76,175.64,-137.08,-46.74
10155200,0.63,0.30,192.184697,240.755969,29.28,57.02,15.23,-0.36
10141000,-2.10,0.23,1169.295938,429.285627,1111.22,280.76,-304.72,-72.97
10126000,-0.31,0.18,1716.296623,1182.889633,306.30,205.39,-39.65,-58.24


In [6]:
def mlp_optimization(search_params, 
                     x_train_scaled_t, 
                     y_train_scaled_t, 
                     loss_func, 
                     model_path, 
                     modelname, 
                     supply, 
                     test_years, 
                     stations,                                                               
                     x_test_temp,
                     x_test_scaled, 
                     y_test_temp,
                     StreamStats,
                     station_index_list):

    epochs, batch_size, learning_rate, decay, L1, L2, L3, L4, L5, L6 = search_params
    
    GS_Eval_DF = pd.DataFrame()
    GS_Eval_dict = {}

    n_models = len(epochs)*len(batch_size)*len(learning_rate)*len(decay)*len(L1)*len(L2)*len(L3)*len(L4)*len(L5)*len(L6)
    print(f"Optimizing the {modelname} model by evaluating {n_models} models using grid search validation")

    counter = 1

    for e in epochs:
        for b in batch_size:
            for lr in learning_rate:
                for d in decay:
                    for l1 in L1:
                        for l2 in L2:
                            for l3 in L3:
                                for l4 in L4:
                                    for l5 in L5:
                                        for l6 in L6:
                                            #Train the model
                                            print(f"Training {counter} of {n_models} models")
                                            layers = x_train_scaled_t.shape[1], l1, l2, l3, l4, l5, l6
                                            params =  lr, d, e, b
                                            
                                            print(f"Parameters: {params}")
                                            print(f"Layers: {layers}")

                                            mlp_model.mlp_train(x_train_scaled_t,
                                                                y_train_scaled_t, 
                                                                layers, params, 
                                                                loss_func, 
                                                                model_path, 
                                                                modelname, 
                                                                shuffle = True)


                                            #Make a prediction for each location, save as compressed pkl file, and send predictions to AWS for use in CSES
                                            Preds_Dict = mlp_model.mlp_predict(test_years, 
                                                                layers, 
                                                                model_path, 
                                                                modelname, 
                                                                stations, 
                                                                x_test_temp,
                                                                x_test_scaled, 
                                                                y_test_temp,
                                                                StreamStats,
                                                                station_index_list)

                                            #Evaluate model performance of the different models, 'flow_cfs_pred', 
                                            prediction_columns = ['NWM_flow', f"{modelname}_flow"]
                                            Eval_DF = Simple_Eval.Simple_Eval(Preds_Dict, 
                                                                            prediction_columns, 
                                                                            modelname, 
                                                                            supply = supply,
                                                                            plots = False, 
                                                                            keystats = False        
                                                                            )

                                            #create dataframe to store key model perf metrics, and inputs
                                            cols = [f"{modelname}_flow_kge", f"{modelname}_flow_rmse", f"{modelname}_flow_mape", f"{modelname}_flow_pbias"]
                                            model_eval = Eval_DF[cols].copy()

                                            #Get mean scoring metrics for AOI - aver kge, mape, pbias
                                            model_eval = pd.DataFrame(model_eval.mean(axis=0)).T

                                            #Add model parameters
                                            parm_dict = {'Epochs': [e],
                                                        'Batchsize': [b],
                                                        'LR': [lr],
                                                        'Decay':[d],
                                                        'L1':[l1],
                                                        'L2':[l2],
                                                        'L3':[l3],
                                                        'L4':[l4],
                                                        'L5':[l5],
                                                        'L6':[l6]}
                                            params_df = pd.DataFrame.from_dict(parm_dict)

                                            #combine model eval df with params df
                                            model_df = pd.concat([model_eval, params_df], axis = 1)
                                            kge = round(model_df[f"{modelname}_flow_kge"].values[0],2)

                                            display(Eval_DF)

                                            #add to overall df
                                            GS_Eval_DF = pd.concat([GS_Eval_DF, model_df])
                                            GS_Eval_DF.sort_values(by = f"{modelname}_flow_kge")
                                            GS_Eval_dict[kge] = Eval_DF
                                            counter = counter +1

    display(GS_Eval_DF)
    return GS_Eval_DF, GS_Eval_dict


In [7]:
epochs = [60,90] # - seems to converge around 80 epochs with 100 batrch size
batch_size = [100]
learning_rate = [0.0001]  #look up learning rate scheduler https://www.geeksforgeeks.org/understanding-pytorch-learning-rate-scheduling/ -  smaller learning rates doing better!
decay = [0.0005]
L1 = [128]
L2 = [128]
L3 = [64]
L4 = [64]
L5 = [32]
L6 = [16]

search_params = epochs, batch_size, learning_rate, decay, L1, L2, L3, L4, L5, L6
loss_func = nn.MSELoss()
supply = False


GS_Eval_DF, GS_Eval_dict = mlp_optimization(search_params, 
                     x_train_scaled_t, 
                     y_train_scaled_t, 
                     loss_func, 
                     model_path, 
                     modelname, 
                     supply, 
                     test_years, 
                     stations,                                                               
                     x_test_temp,
                     x_test_scaled, 
                     y_test_temp,
                     StreamStats,
                     station_index_list)

    



Optimizing the MLP model by evaluating 2 models using grid search validation
Training 1 of 2 models
Parameters: (0.0001, 0.0005, 60, 100)
Layers: (18, 128, 128, 64, 64, 32, 16)


Epochs completed:   0%|          | 0/60 [00:00<?, ?it/s]

In [7]:
GS_DF

,MLP_flow_kge,MLP_flow_rmse,MLP_flow_mape,MLP_flow_pbias,Epochs,Batchsize,LR,Decay,L1,L2,L3,L4,L5,L6
0,0.411667,398.419153,156.715000,-29.308333,"[60, 90]","[50, 100]",[0.0001],[0.0005],[128],[128],[64],[64],[32],[16]
0,0.436667,388.926238,144.631667,-21.055000,"[60, 90]","[50, 100]",[0.0001],[0.0005],[128],[128],[64],[64],[32],[16]
0,0.385000,427.953276,162.625000,-31.746667,"[60, 90]","[50, 100]",[0.0001],[0.0005],[128],[128],[64],[64],[32],[16]
0,0.335000,436.851503,183.781667,-41.878333,"[60, 90]","[50, 100]",[0.0001],[0.0005],[128],[128],[64],[64],[32],[16]


In [1]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(10), desc = 'Progress using tqdm_notebook()'):
    time.sleep(0.5)

Progress using tqdm_notebook():   0%|          | 0/10 [00:00<?, ?it/s]

In [40]:
#building model optimization script
#create dataframe to store key model perf metrics, and inputs
cols = [f"{modelname}_flow_kge", f"{modelname}_flow_rmse", f"{modelname}_flow_mape", f"{modelname}_flow_pbias"]
model_eval = Eval_DF[cols].copy()

#Get mean scoring metrics for AOI - aver kge, mape, pbias
model_eval = pd.DataFrame(model_eval.mean(axis=0)).T

#Add model parameters
parm_dict = {'Epochs': [epochs],
              'Batchsize': [batch_size],
              'LR': [learning_rate],
              'Decay':[decay],
              'L1':[L1],
              'L2':[L2],
              'L3':[L3],
              'L4':[L4],
              'L5':[L5],
              'L6':[L6]}
params_df = pd.DataFrame.from_dict(parm_dict)

#combine model eval df with params df
model_df = pd.concat([model_eval, params_df], axis = 1)

#add to overall df
GS_DF = pd.concat([GS_DF, model_df])




In [41]:
GS_DF

,MLP_flow_kge,MLP_flow_rmse,MLP_flow_mape,MLP_flow_pbias,Epochs,Batchsize,LR,Decay,L1,L2,L3,L4,L5,L6
0,0.368333,419.016979,173.541667,-36.136667,75,100,0.0001,0.0005,128,128,64,64,32,16
0,0.368333,419.016979,173.541667,-36.136667,75,100,0.0001,0.0005,128,128,64,64,32,16
0,0.368333,419.016979,173.541667,-36.136667,75,100,0.0001,0.0005,128,128,64,64,32,16


In [20]:
epochs = 75 # - seems to converge around 80 epochs with 100 batrch size
batch_size = 65
learning_rate = 0.00001  #look up learning rate scheduler https://www.geeksforgeeks.org/understanding-pytorch-learning-rate-scheduling/ -  smaller learning rates doing better!
decay = 0.00005

Eval_DF

,NWM_flow_kge,MLP_flow_kge,NWM_flow_rmse,MLP_flow_rmse,NWM_flow_mape,MLP_flow_mape,NWM_flow_pbias,MLP_flow_pbias,min_storage,max_storage,min_swe,max_swe,min_obs_flow,max_obs_flow,NWM_flow_min,NWM_flow_fmax,MLP_flow_min,MLP_flow_fmax
station_id,,,,,,,,,,,,,,,,,,
10155200,0.63,0.69,192.184697,157.734762,29.28,34.94,15.23,-5.59,60.559554,99.461783,0.0,16.900000,109.958336,1713.95840,57.0,952.0,105.283546,1047.128296
10134500,-0.97,0.67,131.865461,32.215809,545.15,124.77,-173.33,-12.26,54.783838,101.432323,0.0,27.300000,5.775104,188.66667,83.0,389.0,0.000000,173.322296
10140700,0.29,0.53,236.421011,221.738778,221.61,106.01,-30.30,13.72,38.329994,100.776464,0.0,37.300000,15.409375,1754.47910,124.0,972.0,0.000000,1129.556152
10126000,-0.31,0.28,1716.296623,1164.269092,306.30,165.78,-39.65,-54.52,33.856209,97.627451,0.0,18.121429,100.579170,4331.14600,881.0,5460.0,302.933777,4575.572266
10136500,-0.42,0.14,644.133791,450.244341,330.76,127.50,-137.08,-69.43,42.941763,101.853396,0.0,25.062500,28.016666,2272.91670,399.0,2628.0,118.415833,2541.233154
10141000,-2.10,-0.01,1169.295938,501.908287,1111.22,312.48,-304.72,-94.59,42.555835,101.306864,0.0,26.740000,16.032711,3109.79170,603.0,3818.0,75.087067,2842.360596


In [13]:
# epochs = 75 # - seems to converge around 80 epochs with 100 batrch size
# batch_size = 65
# learning_rate = 0.0001  #look up learning rate scheduler https://www.geeksforgeeks.org/understanding-pytorch-learning-rate-scheduling/
# decay = 0.0005
# layers = x_train_scaled_t.shape[1], 128,128,64,64,32,16

Eval_DF

,NWM_flow_kge,MLP_flow_kge,NWM_flow_rmse,MLP_flow_rmse,NWM_flow_mape,MLP_flow_mape,NWM_flow_pbias,MLP_flow_pbias,min_storage,max_storage,min_swe,max_swe,min_obs_flow,max_obs_flow,NWM_flow_min,NWM_flow_fmax,MLP_flow_min,MLP_flow_fmax
station_id,,,,,,,,,,,,,,,,,,
10134500,-0.97,0.59,131.865461,37.120087,545.15,140.60,-173.33,-15.50,54.783838,101.432323,0.0,27.300000,5.775104,188.66667,83.0,389.0,13.527527,181.806870
10140700,0.29,0.52,236.421011,237.958550,221.61,141.13,-30.30,1.07,38.329994,100.776464,0.0,37.300000,15.409375,1754.47910,124.0,972.0,13.735983,1161.830078
10155200,0.63,0.42,192.184697,220.908123,29.28,50.71,15.23,-0.48,60.559554,99.461783,0.0,16.900000,109.958336,1713.95840,57.0,952.0,53.401604,658.170288
10136500,-0.42,0.40,644.133791,377.357272,330.76,171.70,-137.08,-53.15,42.941763,101.853396,0.0,25.062500,28.016666,2272.91670,399.0,2628.0,43.419796,2352.512207
10141000,-2.10,0.25,1169.295938,423.996520,1111.22,260.85,-304.72,-71.17,42.555835,101.306864,0.0,26.740000,16.032711,3109.79170,603.0,3818.0,32.541222,2855.134277
10126000,-0.31,0.21,1716.296623,1176.345525,306.30,197.98,-39.65,-57.46,33.856209,97.627451,0.0,18.121429,100.579170,4331.14600,881.0,5460.0,796.893982,4281.059082


In [9]:
Eval_DF

,NWM_flow_kge,MLP_flow_kge,NWM_flow_rmse,MLP_flow_rmse,NWM_flow_mape,MLP_flow_mape,NWM_flow_pbias,MLP_flow_pbias,min_storage,max_storage,min_swe,max_swe,min_obs_flow,max_obs_flow,NWM_flow_min,NWM_flow_fmax,MLP_flow_min,MLP_flow_fmax
station_id,,,,,,,,,,,,,,,,,,
10140700,0.29,0.50,236.421011,239.406314,221.61,188.50,-30.30,-18.16,38.329994,100.776464,0.0,37.300000,15.409375,1754.47910,124.0,972.0,46.009243,1158.273682
10155200,0.63,0.43,192.184697,220.915754,29.28,57.55,15.23,-9.79,60.559554,99.461783,0.0,16.900000,109.958336,1713.95840,57.0,952.0,88.670708,699.034790
10134500,-0.97,0.35,131.865461,47.451747,545.15,238.03,-173.33,-52.22,54.783838,101.432323,0.0,27.300000,5.775104,188.66667,83.0,389.0,12.334428,196.874176
10136500,-0.42,0.33,644.133791,380.609469,330.76,186.19,-137.08,-62.42,42.941763,101.853396,0.0,25.062500,28.016666,2272.91670,399.0,2628.0,90.487366,2220.493164
10126000,-0.31,0.23,1716.296623,1091.685325,306.30,191.27,-39.65,-51.46,33.856209,97.627451,0.0,18.121429,100.579170,4331.14600,881.0,5460.0,837.900024,3911.201416
10141000,-2.10,0.15,1169.295938,439.946721,1111.22,301.07,-304.72,-82.51,42.555835,101.306864,0.0,26.740000,16.032711,3109.79170,603.0,3818.0,65.584351,2711.665283
